In [10]:
import pandas as pd
import numpy as np
from functions import *

In [11]:
trade = pd.read_csv('data/trade.csv', header=[0,1], index_col=0)
stocks = pd.read_csv('data/stocks.csv')
dk_industry = pd.read_csv('data/dk_industry.csv')

# display(trade.head())
display(stocks.head())
display(dk_industry.head())

,name,Code incl. Expiration,ticker,shares,NACE
0,Aalborg Boldspil,AAB.CO,AAB.CO,2696297.0,93
1,Aarhus Lokalbank,AARHUS.CO^D12 (expired),AARHUS.CO^D12,16203655.0,64
2,Aak Denmark Hldg,AARUTD.CO^K05 (expired),AARUTD.CO^K05,4000000.0,64
3,Affitech,AFFI.CO^J12 (expired),AFFI.CO^J12,487721539.0,72
4,Agat Ejendomme,AGATE.CO,AGATE.CO,117783335.0,68


,NACE,date,Gross output (sales),Intermediate consumption,Investment,Production index,Value-added output
0,1,2001-03-31,36.475,22.890,32.443333,102.755,13.580
1,1,2001-06-30,37.470,23.705,32.403333,104.140,13.765
2,1,2001-09-30,39.600,25.140,32.300000,106.230,14.460
3,1,2001-12-31,37.075,23.500,32.126667,102.785,13.575
4,1,2002-03-31,37.295,23.790,31.433333,100.650,13.505


In [12]:
# Convert the index to datetime (the index holds the dates)
trade.index = pd.to_datetime(trade.index)
df = trade.stack(level=0).reset_index()

# rename columns to have a proper
df.rename(columns={'Timestamp': 'date', 'level_1': 'ticker'}, inplace=True)

# Data Cleaning and Sorting
df.drop_duplicates(inplace=True)
df.dropna(inplace=True) # happens if there was one data point the first day of a given ticker but not the rest of the values (e.g. trade values but no ask or bid)
df.sort_values(['ticker', 'date'], inplace=True)
df.reset_index(drop=True, inplace=True)

# display(df)


/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_36683/1793338663.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = trade.stack(level=0).reset_index()


In [14]:
merged_df = pd.merge(df, stocks, how='left', on=['ticker'])

In [15]:
display(merged_df)

,date,ticker,Ask Close,Bid Close,Trade Close,Trade High,Trade Low,Trade Open,Trade Volume,Turnover,name,Code incl. Expiration,shares,NACE
0,2001-01-31,AAB.CO,4183.493029,4026.612041,4026.612041,4183.493029,2614.683143,2771.564132,6.915369e+02,2.255122e+06,Aalborg Boldspil,AAB.CO,2696297.0,93
1,2001-02-28,AAB.CO,3765.143726,3660.556401,3765.143726,4235.786692,3399.088086,4131.199367,5.061034e+02,2.047656e+06,Aalborg Boldspil,AAB.CO,2696297.0,93
2,2001-03-31,AAB.CO,3660.556401,3529.822244,3503.675412,3765.143726,3451.381749,3765.143726,1.172800e+02,4.385660e+05,Aalborg Boldspil,AAB.CO,2696297.0,93
3,2001-04-30,AAB.CO,3346.794423,3189.913435,3189.913435,4183.493029,3189.913435,4183.493029,1.263059e+02,4.467180e+05,Aalborg Boldspil,AAB.CO,2696297.0,93
4,2001-05-31,AAB.CO,3137.619772,3006.885615,2876.151458,3399.088086,2876.151458,3137.619772,1.653355e+02,5.048560e+05,Aalborg Boldspil,AAB.CO,2696297.0,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48803,2024-11-30,ZELA.CO,730.000000,729.000000,730.000000,893.000000,672.500000,789.500000,4.650485e+06,3.587543e+09,Zealand Pharma,ZELA.CO,71051296.0,72
48804,2024-12-31,ZELA.CO,716.500000,715.500000,715.500000,819.000000,587.000000,728.000000,4.138727e+06,2.995576e+09,Zealand Pharma,ZELA.CO,71051296.0,72
48805,2025-01-31,ZELA.CO,738.000000,737.000000,735.500000,808.000000,682.500000,718.500000,3.538390e+06,2.572094e+09,Zealand Pharma,ZELA.CO,71051296.0,72
48806,2025-02-28,ZELA.CO,665.000000,664.000000,663.000000,788.000000,657.000000,714.500000,3.876841e+06,2.801496e+09,Zealand Pharma,ZELA.CO,71051296.0,72


In [3]:
dk_industry['date'] = pd.to_datetime(dk_industry['date'])

# join stocks and dk_industry on 'NACE industry' 
stocks = stocks.merge(dk_industry, how='left', left_on='NACE', right_on='NACE')
display(stocks.head())


,name,Code incl. Expiration,ticker,shares,NACE,date,Gross output (sales),Intermediate consumption,Investment,Production index,Value-added output
0,Aalborg Boldspil,AAB.CO,AAB.CO,2696297.0,93,2001-03-31,253.6575,85.556667,43.852,92.026667,131.283333
1,Aalborg Boldspil,AAB.CO,AAB.CO,2696297.0,93,2001-06-30,255.8500,86.470000,43.740,93.223333,132.116667
2,Aalborg Boldspil,AAB.CO,AAB.CO,2696297.0,93,2001-09-30,256.7075,87.070000,43.600,92.783333,132.440000
3,Aalborg Boldspil,AAB.CO,AAB.CO,2696297.0,93,2001-12-31,260.8300,88.216667,43.424,93.260000,134.206667
4,Aalborg Boldspil,AAB.CO,AAB.CO,2696297.0,93,2002-03-31,258.1550,87.503333,42.880,92.626667,133.146667


In [ ]:
merged_df = pd.merge(df, stocks, how='left', on=['date', 'ticker'])
display(merged_df)


# for each 


,date,ticker,Ask Close,Bid Close,Trade Close,Trade High,Trade Low,Trade Open,Trade Volume,Turnover,name,Code incl. Expiration,shares,NACE,Gross output (sales),Intermediate consumption,Investment,Production index,Value-added output
0,2001-01-31,AAB.CO,4183.493029,4026.612041,4026.612041,4183.493029,2614.683143,2771.564132,6.915369e+02,2.255122e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2001-02-28,AAB.CO,3765.143726,3660.556401,3765.143726,4235.786692,3399.088086,4131.199367,5.061034e+02,2.047656e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001-03-31,AAB.CO,3660.556401,3529.822244,3503.675412,3765.143726,3451.381749,3765.143726,1.172800e+02,4.385660e+05,Aalborg Boldspil,AAB.CO,2696297.0,93.0,253.6575,85.556667,43.852000,92.026667,131.283333
3,2001-04-30,AAB.CO,3346.794423,3189.913435,3189.913435,4183.493029,3189.913435,4183.493029,1.263059e+02,4.467180e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001-05-31,AAB.CO,3137.619772,3006.885615,2876.151458,3399.088086,2876.151458,3137.619772,1.653355e+02,5.048560e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48803,2024-11-30,ZELA.CO,730.000000,729.000000,730.000000,893.000000,672.500000,789.500000,4.650485e+06,3.587543e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48804,2024-12-31,ZELA.CO,716.500000,715.500000,715.500000,819.000000,587.000000,728.000000,4.138727e+06,2.995576e+09,Zealand Pharma,ZELA.CO,71051296.0,72.0,500.3500,220.995000,63.998333,123.325000,171.965000
48805,2025-01-31,ZELA.CO,738.000000,737.000000,735.500000,808.000000,682.500000,718.500000,3.538390e+06,2.572094e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48806,2025-02-28,ZELA.CO,665.000000,664.000000,663.000000,788.000000,657.000000,714.500000,3.876841e+06,2.801496e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
